In [1]:
import os
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from preprocess_utility import *
from data_manager import *
from metadata import *

Setting environment for Gordon


No vtk


In [ ]:
for stack in all_stacks:
    
# for stack in ['MD603']:
        
    first_sec, last_sec = metadata_cache['section_limits'][stack]

    #################################

#     t = time.time()
#     sys.stderr.write('running svm classifier ...')

#     exclude_nodes = [33]

#     run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d' % \
#                     {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/svm_predict.py',
#                     'stack': stack},
#                     kwargs_list=dict(sections=range(first_sec, last_sec+1)),
#                     exclude_nodes=exclude_nodes,
#                     argument_type='partition')

#     sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

#     #################################

    t = time.time()
    sys.stderr.write('interpolating scoremaps ...')

    exclude_nodes = [33]

    run_distributed4(command='%(script_path)s %(stack)s %%(first_sec)d %%(last_sec)d' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/interpolate_scoremaps_v2.py',
                    'stack': stack},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # ~240 seconds 

    #################################

    t = time.time()
    sys.stderr.write('visualize scoremaps ...')

    add_annotation = False

    exclude_nodes = [33]
    # first_sec, last_sec = DataManager.load_cropbox(stack)[4:]

    run_distributed4(command='%(script_path)s %(stack)s -b %%(first_sec)d -e %%(last_sec)d %(add_annotation)s' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/visualize_scoremaps_v2.py',
                    'stack': stack,
                    'add_annotation': '-a' if add_annotation else ''},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 464 seconds / stack

    #################################

    paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                        'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
    singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
    structures = paired_structures + singular_structures

    #################################

    t = time.time()
    sys.stderr.write('constructing score volumes ...')

    exclude_nodes = [33]

    run_distributed4(command='%(script_path)s %(stack)s %%(label)s' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'reconstruct') + '/construct_score_volume_v2.py',
                    'stack': stack},
                    kwargs_list=dict(label=structures),
                    exclude_nodes=exclude_nodes,
                    argument_type='single')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t))

    #################################

    downscale_factor = 32

    #################################

    print stack

    volume_allLabels = {}

    for name_u in structures:
        volume = DataManager.load_score_volume(stack, label=name_u, downscale=downscale_factor, train_sample_scheme=1)
        volume_allLabels[name_u] = volume
#         del volume

    t1 = time.time()

    gradient_dir = create_if_not_exists(os.path.join(VOLUME_ROOTDIR, stack, 'score_volume_gradients'))

    for name_u in structures:

        t = time.time()

        gy_gx_gz = np.gradient(volume_allLabels[name_u].astype(np.float16), 3, 3, 3) 
        # 3.3 second - re-computing is much faster than loading

        sys.stderr.write('Gradient %s: %f seconds\n' % (name_u, time.time() - t))

        t = time.time()

        bp.pack_ndarray_file(gy_gx_gz[0], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gy.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
        bp.pack_ndarray_file(gy_gx_gz[1], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gx.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
        bp.pack_ndarray_file(gy_gx_gz[2], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gz.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))

        del gy_gx_gz

        sys.stderr.write('save %s: %f seconds\n' % (name_u, time.time() - t))


    sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))

interpolating scoremaps ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


In [4]:
paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                    'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
structures = paired_structures + singular_structures

In [10]:
for stack in all_stacks:    
# for stack in ['MD603']:
        
    first_sec, last_sec = metadata_cache['section_limits'][stack]


    downscale_factor = 32

    #################################

    print stack

    volume_allLabels = {}

    for name_u in structures:
        volume = DataManager.load_score_volume(stack, label=name_u, downscale=downscale_factor, train_sample_scheme=1)
        volume_allLabels[name_u] = volume
#         del volume

    t1 = time.time()

    gradient_dir = create_if_not_exists(os.path.join(VOLUME_ROOTDIR, stack, 'score_volume_gradients'))

    for name_u in structures:

        t = time.time()

        gy_gx_gz = np.gradient(volume_allLabels[name_u].astype(np.float32), 3, 3, 3) 
        # .astype(np.float32) is important; 
        # Otherwise the score volume is type np.float16, np.gradient requires np.float32 and will have to convert which is very slow
        # 20s (float32) vs. 2s (float16)
        # 3.3 second - re-computing is much faster than loading

        sys.stderr.write('Gradient %s: %f seconds\n' % (name_u, time.time() - t))

        t = time.time()

        bp.pack_ndarray_file(gy_gx_gz[0].astype(np.float16), os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gy.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
        bp.pack_ndarray_file(gy_gx_gz[1].astype(np.float16), os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gx.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
        bp.pack_ndarray_file(gy_gx_gz[2].astype(np.float16), os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gz.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
#         bp.pack_ndarray_file(gy_gx_gz[0], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gy.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
#         bp.pack_ndarray_file(gy_gx_gz[1], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gx.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
#         bp.pack_ndarray_file(gy_gx_gz[2], os.path.join(gradient_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s_trainSampleScheme_1_gz.bp' % {'stack':stack, 'label':name_u, 'ds': downscale_factor}))
        # output of gradient is float32; if convert from float32 to float16, 4s; if save as float32, 2s

        del gy_gx_gz

        sys.stderr.write('save %s: %f seconds\n' % (name_u, time.time() - t))


    sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))

MD589


Gradient 5N: 1.736946 seconds
save 5N: 2.792673 seconds
Gradient 6N: 1.709873 seconds
save 6N: 2.062216 seconds
Gradient 7N: 1.774315 seconds
save 7N: 2.879442 seconds
Gradient 7n: 1.726864 seconds
save 7n: 2.451032 seconds
Gradient Amb: 1.731376 seconds
save Amb: 2.724903 seconds
Gradient LC: 1.722921 seconds
save LC: 2.446536 seconds
Gradient LRt: 1.778243 seconds
save LRt: 2.939850 seconds
Gradient Pn: 1.759767 seconds
save Pn: 2.811315 seconds
Gradient Tz: 1.727292 seconds
save Tz: 2.288719 seconds
Gradient VLL: 1.771286 seconds
save VLL: 2.863430 seconds
Gradient RMC: 1.803575 seconds
save RMC: 3.050235 seconds
Gradient SNC: 1.756067 seconds
save SNC: 2.922825 seconds
Gradient SNR: 1.762315 seconds
save SNR: 2.801711 seconds
Gradient 3N: 1.781714 seconds
save 3N: 2.915415 seconds
Gradient 4N: 1.732980 seconds
save 4N: 2.405576 seconds
Gradient Sp5I: 1.769540 seconds
save Sp5I: 2.833300 seconds
Gradient Sp5O: 1.747406 seconds
save Sp5O: 2.884168 seconds
Gradient Sp5C: 1.765377 seco

MD594


Gradient 5N: 1.761571 seconds
save 5N: 2.859894 seconds
Gradient 6N: 1.721962 seconds
save 6N: 2.254084 seconds
Gradient 7N: 1.772419 seconds
save 7N: 2.949624 seconds
Gradient 7n: 1.724967 seconds
save 7n: 2.403525 seconds
Gradient Amb: 1.763838 seconds
save Amb: 2.852284 seconds
Gradient LC: 1.736353 seconds
save LC: 2.623500 seconds
Gradient LRt: 1.797878 seconds
save LRt: 3.060683 seconds
Gradient Pn: 1.771339 seconds
save Pn: 2.885558 seconds
Gradient Tz: 1.744933 seconds
save Tz: 2.496955 seconds
Gradient VLL: 1.767515 seconds
save VLL: 2.888929 seconds
Gradient RMC: 1.785297 seconds
save RMC: 3.087179 seconds
Gradient SNC: 1.768346 seconds
save SNC: 3.105579 seconds
Gradient SNR: 1.777976 seconds
save SNR: 2.889912 seconds
Gradient 3N: 1.771332 seconds
save 3N: 2.845661 seconds
Gradient 4N: 1.731234 seconds
save 4N: 2.479955 seconds
Gradient Sp5I: 1.759675 seconds
save Sp5I: 2.904405 seconds
Gradient Sp5O: 1.778940 seconds
save Sp5O: 2.930579 seconds
Gradient Sp5C: 1.760503 seco

MD593


...

Gradient 5N: 2.103974 seconds
save 5N: 3.395426 seconds
Gradient 6N: 2.029707 seconds
save 6N: 2.672484 seconds
Gradient 7N: 2.095262 seconds
save 7N: 3.449002 seconds
Gradient 7n: 2.030564 seconds
save 7n: 2.891300 seconds
Gradient Amb: 2.080706 seconds
save Amb: 3.224901 seconds
Gradient LC: 2.066066 seconds
save LC: 3.037190 seconds
Gradient LRt: 2.124397 seconds
save LRt: 3.583338 seconds
Gradient Pn: 2.068641 seconds
save Pn: 3.365857 seconds
Gradient Tz: 2.060656 seconds
save Tz: 2.920608 seconds
Gradient VLL: 2.085238 seconds
save VLL: 3.386102 seconds
Gradient RMC: 2.113681 seconds
save RMC: 3.604349 seconds
Gradient SNC: 2.132623 seconds
save SNC: 3.621465 seconds
Gradient SNR: 2.091290 seconds
save SNR: 3.348930 seconds
Gradient 3N: 2.106267 seconds
save 3N: 3.426158 seconds
Gradient 4N: 2.060007 seconds
save 4N: 2.928766 seconds
Gradient Sp5I: 2.100080 seconds
save Sp5I: 3.357200 seconds
Gradient Sp5O: 2.073799 seconds
save Sp5O: 3.383887 seconds
Gradient Sp5C: 2.063627 seco

MD602


Gradient 5N: 2.429364 seconds
save 5N: 3.559368 seconds
Gradient 6N: 2.325621 seconds
save 6N: 2.891843 seconds
Gradient 7N: 2.394951 seconds
save 7N: 3.674492 seconds
Gradient 7n: 2.365790 seconds
save 7n: 3.173763 seconds
Gradient Amb: 2.403849 seconds
save Amb: 3.658761 seconds
Gradient LC: 2.364408 seconds
save LC: 3.361756 seconds
Gradient LRt: 2.405932 seconds
save LRt: 3.814688 seconds
Gradient Pn: 2.382398 seconds
save Pn: 3.674933 seconds
Gradient Tz: 2.355418 seconds
save Tz: 3.218896 seconds
Gradient VLL: 2.366723 seconds
save VLL: 3.710066 seconds
Gradient RMC: 2.408611 seconds
save RMC: 3.836341 seconds
Gradient SNC: 2.413864 seconds
save SNC: 3.792380 seconds
Gradient SNR: 2.399139 seconds
save SNR: 3.543487 seconds
Gradient 3N: 2.418092 seconds
save 3N: 3.775618 seconds
Gradient 4N: 2.342564 seconds
save 4N: 3.152490 seconds
Gradient Sp5I: 2.377887 seconds
save Sp5I: 3.688663 seconds
Gradient Sp5O: 2.400057 seconds
save Sp5O: 3.692232 seconds
Gradient Sp5C: 2.397406 seco

MD603


Gradient 5N: 2.742858 seconds
save 5N: 4.049058 seconds
Gradient 6N: 2.688832 seconds
save 6N: 3.305243 seconds
Gradient 7N: 2.752258 seconds
save 7N: 4.172722 seconds
Gradient 7n: 2.725478 seconds
save 7n: 3.817498 seconds
Gradient Amb: 2.739412 seconds
save Amb: 3.975930 seconds
Gradient LC: 2.712381 seconds
save LC: 3.698589 seconds
Gradient LRt: 2.767523 seconds
save LRt: 4.228984 seconds
Gradient Pn: 2.760741 seconds
save Pn: 4.092969 seconds
Gradient Tz: 2.691181 seconds
save Tz: 3.508600 seconds
Gradient VLL: 2.741633 seconds
save VLL: 4.124942 seconds
Gradient RMC: 2.785208 seconds
save RMC: 4.305143 seconds
Gradient SNC: 2.762551 seconds
save SNC: 4.220144 seconds
Gradient SNR: 2.735227 seconds
save SNR: 3.960894 seconds
Gradient 3N: 2.772699 seconds
save 3N: 4.264593 seconds
Gradient 4N: 2.734383 seconds
save 4N: 3.794864 seconds
Gradient Sp5I: 2.745808 seconds
save Sp5I: 4.157872 seconds
Gradient Sp5O: 2.755927 seconds
save Sp5O: 4.160333 seconds
Gradient Sp5C: 2.761328 seco

In [ ]:
for stack in all_stacks:
    first_sec, last_sec = metadata_cache['section_limits'][stack]
    
    t = time.time()
    sys.stderr.write('visualize scoremaps ...')

    add_annotation = False

    exclude_nodes = [33]

    run_distributed4(command='%(script_path)s %(stack)s -b %%(first_sec)d -e %%(last_sec)d %(add_annotation)s' % \
                    {'script_path': os.path.join(os.environ['REPO_DIR'], 'learning') + '/visualize_scoremaps_v2.py',
                    'stack': stack,
                    'add_annotation': '-a' if add_annotation else ''},
                    kwargs_list=dict(sections=range(first_sec, last_sec+1)),
                    exclude_nodes=exclude_nodes,
                    argument_type='partition')

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 464 seconds / stack

visualize scoremaps ...

['gcn-20-33.sdsc.edu'] are excluded


# Align

In [4]:
t = time.time()
sys.stderr.write('align all subjects to atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/align_subject_brain_to_atlas_v2.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t))  # 526 seconds

align all subjects to atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 526.264969 seconds


In [6]:
t = time.time()
sys.stderr.write('transform atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/transform_brains.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

transform atlas ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 270.454302 seconds


In [ ]:
t = time.time()
sys.stderr.write('visualize aligned atlas overlay ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 0 atlasV2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/visualize_aligned_brains_v2.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 310 seconds

In [ ]:
t = time.time()
sys.stderr.write('align all subjects to atlas ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/align_subject_brain_to_atlas_v2.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

In [ ]:
t = time.time()
sys.stderr.write('LOCAL align ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 1' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/fit_atlas_structure_to_subject_v2.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

LOCAL align ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


In [3]:
t = time.time()
sys.stderr.write('LOCAL align ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 2' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/fit_atlas_structure_to_subject_v2.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

LOCAL align ...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 9022.209641 seconds


In [ ]:
t = time.time()
sys.stderr.write('LOCAL align ...')

exclude_nodes = [33]

run_distributed4(command='%(script_path)s %%(stack)s 1 1 3' % \
                {'script_path': os.path.join(os.environ['REPO_DIR'], 'registration') + '/fit_atlas_structure_to_subject_v2.py'},
                kwargs_list=dict(stack=all_stacks),
                exclude_nodes=exclude_nodes,
                argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) 

LOCAL align ...

['gcn-20-33.sdsc.edu'] are excluded
